In [1]:
import os
import json

from io import BytesIO

from urllib.parse import urlparse
from urllib.parse import parse_qs

import requests

import numpy as np
import pandas as pd

from PIL import Image
from jinja2 import Template

In [2]:
folder = '10-kubernetes'

In [3]:
os.makedirs(folder + '/images', exist_ok=True)

In [4]:
with open(folder + '/meta.json') as f_in:
    data = json.load(f_in)

In [5]:
session = data['session']
session_name = data['name']

In [6]:
df = pd.read_csv(folder + '/' + data['data'])

In [7]:
df = df.fillna('')

In [8]:
df

,lesson,name,page_name,video,slides,notebook
0,1,Overview,01-overview.md,https://www.youtube.com/watch?v=mvPER7YfTkw,https://www.slideshare.net/AlexeyGrigorev/ml-z...,
1,2,TensorFlow Serving,02-tensorflow-serving.md,https://www.youtube.com/watch?v=deXR2fThYDw,,
2,3,Creating a pre-processing service,03-preprocessing.md,https://www.youtube.com/watch?v=OIlrS14Zi0o,,code/tf-serving-connect.ipynb
3,4,Running everything locally with Docker-compose,04-docker-compose.md,https://www.youtube.com/watch?v=ZhQQfpWfkKY,https://www.slideshare.net/AlexeyGrigorev/ml-z...,
4,5,Introduction to Kubernetes,05-kubernetes-intro.md,https://www.youtube.com/watch?v=UjVkpszDzgk,https://www.slideshare.net/AlexeyGrigorev/ml-z...,
5,6,Deploying a simple service to Kubernetes,06-kubernetes-simple-service.md,https://www.youtube.com/watch?v=PPUCVRIV9t8,https://www.slideshare.net/AlexeyGrigorev/ml-z...,
6,7,Deploying TensorFlow models to Kubernetes,07-kubernetes-tf-serving.md,NOT_READY,,
7,8,Deploying to EKS,08-eks.md,NOT_READY,,
8,9,Summary,09-summary.md,NOT_READY,,
9,10,Explore more,10-explore-more.md,,,


In [9]:
template_string = """
## {{ session }}.{{ lesson }} {{ name }}

{% if video %}<a href="{{ video }}"><img src="{{ thumbnail }}"></a>{% endif %}
{% if coming_soon %}COMING SOON{% endif %} 

{% if slides %}[Slides]({{ slides }}){% endif %}


## Notes

Add notes from the video (PRs are welcome)


<table>
   <tr>
      <td>⚠️</td>
      <td>
         The notes are written by the community. <br>
         If you see an error here, please create a PR with a fix.
      </td>
   </tr>
</table>


## Navigation

{{ navigation }}
"""

In [10]:
def download_thumbnail(video, session, lesson):
    if type(lesson) in [np.int64]:
        thumbnail_file = 'thumbnail-%d-%02d.jpg' % (session, lesson)
    else:
        thumbnail_file = 'thumbnail-%d-%s.jpg' % (session, lesson)
    thumbnail_file_full = '%s/images/%s' % (folder, thumbnail_file)

    if os.path.exists(thumbnail_file_full):
        print(thumbnail_file_full, 'exists')
        return 'images/%s' % thumbnail_file

    video_id = parse_qs(urlparse(video).query)['v'][0]
    print('processing video', video_id)
    thumbnail_url = 'https://img.youtube.com/vi/%s/0.jpg' % video_id

    response = requests.get(thumbnail_url)
    thumbnail = Image.open(BytesIO(response.content))
    w_img, h_img = thumbnail.size

    play = Image.open('../images/play.png')
    w_play, h_play = play.size
    
    x0 = w_img // 2 - w_play // 2
    y0 = h_img // 2 - h_play // 2

    thumbnail.paste(play, (x0, y0), play)
    thumbnail.save(thumbnail_file_full, quality=90)

    print('saved to', thumbnail_file_full)

    return 'images/%s' % thumbnail_file

In [12]:
create_files = True

for i in range(len(df)):
    row = df.loc[i]
    params = row.to_dict()
    params['session'] = session

    navigation = [
        '* [Machine Learning Zoomcamp course](../)',
        '* [Session %s: %s](./)' % (session, session_name) 
    ]

    if i > 0:
        prev_row = df.loc[i - 1]
        line = '* Previous: [%s](%s)' % (prev_row['name'], prev_row.page_name)
        navigation.append(line)

    if i + 1 < len(df):
        next_row = df.loc[i + 1]
        line = '* Next: [%s](%s)' % (next_row['name'], next_row.page_name)
        navigation.append(line)
        
    params['navigation'] = '\n'.join(navigation)
    
    if row.video and row.video != 'NOT_READY':
        thumbnail = download_thumbnail(row.video, session, row.lesson)
        params['thumbnail'] = thumbnail

    if row.video == 'NOT_READY':
        del params['video']
        params['coming_soon'] = True
    
    if 'video' in params and len(params['video']) > 0:
        params['video'] = params['video'] + '&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR'

    template = Template(template_string)
    rendered = template.render(**params)
    print(rendered)
    
    page_full = '%s/%s' % (folder, row.page_name)
    
    print()

    if create_files:
        if (not os.path.exists(page_full)) or os.path.getsize(page_full) <= 1:
            with open(page_full, 'w', encoding='utf-8') as f_out:
                f_out.write(rendered)
            print("created", page_full)
        else:
            with open(page_full, 'a', encoding='utf-8') as f_out:
                f_out.write('\n\n========\n\n')
                f_out.write(rendered)
            print("APPENDED TO", page_full)
    
    print()
    print()

10-kubernetes/images/thumbnail-10-01.jpg exists

## 10.1 Overview

<a href="https://www.youtube.com/watch?v=mvPER7YfTkw&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR"><img src="images/thumbnail-10-01.jpg"></a>
 

[Slides](https://www.slideshare.net/AlexeyGrigorev/ml-zoomcamp-10-kubernetes)


## Notes

Add notes from the video (PRs are welcome)


<table>
   <tr>
      <td>⚠️</td>
      <td>
         The notes are written by the community. <br>
         If you see an error here, please create a PR with a fix.
      </td>
   </tr>
</table>


## Navigation

* [Machine Learning Zoomcamp course](../)
* [Session 10: Kubernetes and TensorFlow Serving](./)
* Next: [TensorFlow Serving](02-tensorflow-serving.md)

created 10-kubernetes/01-overview.md


10-kubernetes/images/thumbnail-10-02.jpg exists

## 10.2 TensorFlow Serving

<a href="https://www.youtube.com/watch?v=deXR2fThYDw&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR"><img src="images/thumbnail-10-02.jpg"></a>
 




## Notes

Add notes from the video 